In [0]:
%sql
create schema datamodel.silver;

#### Transformation

In [0]:
spark.sql("""
            select 
                *,
                split(customer_email, '@')[1] as customer_email_host,
                current_date() as processed_date
            from datamodel.bronze.bronze_tbl
          """).createOrReplaceTempView("silver_src")

#### Silver Table Creation & MERGE/UPSERT

In [0]:
if spark.catalog.tableExists("datamodel.silver.silver_tbl"):
    spark.sql("""
              merge into datamodel.silver.silver_tbl as dst
              using silver_src as src
              on dst.order_id = src.order_id
              when matched then
                update set *
              when not matched then
                insert *
              """)
else:
    spark.sql("""
                create table datamodel.silver.silver_tbl
                as
                select 
                    *
                from silver_src
              """)

In [0]:
%sql
select * from datamodel.silver.silver_tbl;

order_id,order_date,customer_id,customer_name,customer_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated,customer_email_host,processed_date
1001,2024-07-01,1,Alice Johnson,alice@gmail.com,501,iPhone 14,Electronics,1,999.99,Credit Card,USA,2024-07-01,gmail.com,2025-08-11
1002,2024-07-01,2,Bob Smith,bob@yahoo.com,502,AirPods Pro,Electronics,2,199.99,PayPal,USA,2024-07-01,yahoo.com,2025-08-11
1003,2024-07-01,3,Charlie Brown,charlie@outlook.com,503,Nike Shoes,Footwear,1,129.99,Credit Card,Canada,2024-07-01,outlook.com,2025-08-11
1004,2024-07-02,4,David Lee,david@abc.com,504,Samsung S23,Electronics,1,899.99,Credit Card,USA,2024-07-02,abc.com,2025-08-11
1005,2024-07-02,1,Alice Johnson,alice@gmail.com,503,Nike Shoes,Footwear,2,129.99,Credit Card,USA,2024-07-02,gmail.com,2025-08-11
